### Coding Attention Mechanism

In [2]:
from importlib.metadata import version

print("torch version:", version("torch"))

torch version: 2.3.0


Simple attention mechanism 

For Query, Key, and Value vectors, we use the same vector for simplicity.

```python

In [3]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [4]:
query = inputs[1]
print("query:", query)

query: tensor([0.5500, 0.8700, 0.6600])


In [5]:
attention_scores_2 = torch.empty(inputs.shape[0], dtype=torch.float32)
print("attention_scores_2:", attention_scores_2)

attention_scores_2: tensor([0., 0., 0., 0., 0., 0.])


In [6]:
for i, x in enumerate(inputs):
    attention_scores_2[i] = torch.dot(query, x)

print("attention_scores_2:", attention_scores_2)

attention_scores_2: tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


Above weights are not standerdized.

In [7]:
# standerdizing weights

attention_scores_norm = attention_scores_2/ attention_scores_2.sum()
print("attention_scores_norm:", attention_scores_norm)

# usually use softmax function for normalize

print("Attention scores sum:", attention_scores_norm.sum())

attention_scores_norm: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Attention scores sum: tensor(1.0000)


In [8]:
attention_weights_softmax = torch.softmax(attention_scores_2, dim=0)

print("Attention weights:",attention_weights_softmax)
print("Sum:", attention_scores_norm.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.0000)


In [9]:
# as final step let's calculate the context vector

context_vec = torch.zeros(query.shape)
print(context_vec)

for i, x in enumerate(inputs):
    context_vec += attention_scores_norm[i]*x
    print(f"context vec {i} step:", 
    context_vec)

print(context_vec)

tensor([0., 0., 0.])
context vec 0 step: tensor([0.0625, 0.0218, 0.1295])
context vec 1 step: tensor([0.1879, 0.2200, 0.2798])
context vec 2 step: tensor([0.3160, 0.4112, 0.4237])
context vec 3 step: tensor([0.3443, 0.4857, 0.4661])
context vec 4 step: tensor([0.4273, 0.5126, 0.4769])
context vec 5 step: tensor([0.4355, 0.6451, 0.5680])
tensor([0.4355, 0.6451, 0.5680])


Computing attention scores for all input tokens

In [10]:
attention_scores = torch.empty(6,6)

print(attention_scores)

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])


In [11]:
for i, xi in enumerate(inputs):
    for j, xj in enumerate(inputs):
        attention_scores[i, j] = torch.dot(xi,xj)


print(attention_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [12]:
torch.dot(inputs[0],inputs[0])

tensor(0.9995)

* dot product: Returns a scalar that is the sum of the products of corresponding elements.
* '*' : element-wise multiplication

Can achieve the same result with `torch.matmul` or `torch.bmm` (batch matrix multiplication).

In [13]:
atten_scores_matrix_mul = inputs @ inputs.T
print(atten_scores_matrix_mul)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


Above code can be confusing because it uses the same vector for Query, Key, and Value. Let's use trainable weights and properly compute the attention scores.

### Implementing Self-Attention (Trainable Weights) : IMPORTANT

In [14]:
x_2 = inputs[1]
d_in = inputs.shape[1] # the input embedding s
d_out = 2 # the output embedding size

In [21]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [24]:
# create query, key, value vectors

query_2 = x_2 @ W_query # _2 because it's with respect to the 2nd input element
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

In [25]:
print(value_2)

tensor([0.3951, 1.0037])


In [26]:
keys = inputs @ W_key
values = inputs @ W_value

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


In [27]:
attention_scores_2 = query_2 @ keys.T # All attention scores for given query

print(attention_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [29]:
d_k = keys.shape[1]

attn_weights_2 = torch.softmax(attention_scores_2 / d_k**0.5, dim=-1 )
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


In [30]:
context_vec_2 = attn_weights_2 @ values

print(context_vec_2)

tensor([0.3061, 0.8210])


### Implementing a compact SelfAttention class